In [1]:
from search_2 import *
import time

N = 100

# First we code up the search agent
class VacuumSearch(Problem):
    def __init__(self, initial=(0,0), goal=None, dirt=[], **kwds):
        Problem.__init__(self, initial=initial, goal=goal, dirt=dirt, **kwds)
        self.dirt = dirt
        
    directions = [(-1, 0), (+1, 0), (0, +1), (0, -1)]

    def action_cost(self, s, action, s1):
        if s == s1:
            return -100
        return 1
    
    def is_goal(self, state):
        return self.dirt == []

    def result(self, state, action):
        return action
    
    def actions(self, state):
        if state in self.dirt:
            self.dirt.remove(state)
            S = {(state[0],state[1])}
            return S
        x, y = state
        S = {(x + dx, y + dy) for (dx, dy) in self.directions}
        Z = {x for x in S if x[0] in range(N) and x[1] in range(N)}
        return Z


k = int(0.2 * N)
dirt = [(x,y) for x in range(N) for y in range(N)]
dirt = random.sample(dirt, k=k)
v = VacuumSearch(dirt=dirt)

# execution time
t0 = time.time()
u1 = uniform_cost_search(v)
t1 = time.time()
print(t1-t0)

# path cost of actions
print(u1.path_cost) 

0.5227794647216797
-1525


In [3]:
from agents import *
import numpy as np

class VacuumAgent(Agent):

    def __init__(self, program = None):
        super().__init__(program)
        self.location = [0, 0]
        self.direction = 'R' # 'R', 'L', 'U', 'D'

# The agent should have no knowledge of the dirt distribution.
# The environment has the dirt distribution. 
def program(percept):
    if percept:
        return ('Suck','None')
    else:
        bump = 'Bump' if agent.bump else 'None'
        return ('Move', bump)

class Environment:
    
    def __init__(self):
        self.agents = []
        
    def percept(self, agent):
        loc = agent.location
        
        return 

    def execute_action(self, agent, action):
        if action[0] == 'Suck':
            self.floor_plan[agent.location[0]][agent.location[1]] = 1
            agent.performance += 100
        if action[0] == 'Move':
            agent.performance -= 1
            if action[1] == 'Bump':
                agent.direction = random.choice(['R', 'L', 'U', 'D'])
                agent.bump = False
            else:
                agent.bump = self.move_agent(agent)

    def move_agent(self, agent):
        loc = agent.location
        if agent.direction == 'R':
            if loc[1] + 1 == N:
                agent.bump = True
            else:
                agent.location[1] += 1
        elif agent.direction == 'L':
            if loc[1] - 1 == 0:
                agent.bump = True
            else:
                agent.location[1] -= 1
        elif agent.direction == 'U':
            if loc[0] - 1 == 0:
                agent.bump = True
            else:
                agent.location[0] -= 1
        elif agent.direction == 'D':
            if loc[0] + 1 == N:
                agent.bump = True
            else:
                agent.location[0] += 1
        return agent.bump

    def step(self):
        action = agent.program(self.percept(agent))
        self.execute_action(self.agents[0], action)
        print(action, self.agents[0].location, self.agents[0].direction)

    def run(self,steps=100):
        for step in range(steps):
            self.step()
    
    def display_floor(self):
        print(self.floor_plan)

    def add_agent(self, agent):
        self.agents.append(agent)


env = Environment()
agent = VacuumAgent(program)
env.add_agent(agent)

env.display_floor()
env.run()
env.display_floor()
print(agent.performance)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 2 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 0 1 0 1 0 2 2 2 2 2 2 0]
 [0 0 1 0 1 1 2 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
('Move', 'None') [1, 2] R
('Move', 'None') [1, 3] R
('Suck', 'None') [1, 3] R
('Move', 'None') [1, 4] R
('Move', 'None') [1, 5] R
('Move', 'None') [1, 6] R
('Move', 'None') [1, 7] R
('Move', 'None') [1, 8] R
('Move', 'None') [1, 9] R
('Move', 'None') [1, 10] R
('Move', 'None') [1, 11] R
('Move', 'None') [1, 12] R
('Move', 'None') [1, 13] R
('Move', 'None') [1, 13] R
('Move', 'Bump') [1, 13] D
('Move', 'None') [2, 13] D
('Suck', 'None') [2, 13] D
('Move', 'None') [2, 13] D
('Move', 'Bump') [2, 13] R
('Move', 'None') [2, 13] R
('Move', 'Bump') [2, 13] R
('Move', 'None') [2, 13] R
('Move', 'Bump') [2, 13] D
('Move', 'None') [2, 13] D
('Move', 'Bump') [2, 13] D
('Move', 'None') [2, 13] D
('Move', 'Bump') [2, 13] U
('Move', 'None') [1, 13] U
('Move', 'None') [1, 13] U
('Move', 'Bump') [1, 13] U
('Move', 'None') [1, 13] U
('Move